In [ ]:
!pip install tensorflow-text==2.5
!pip install tf-models-official==2.5

In [ ]:
import os
import shutil
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import scipy.interpolate as interp
from collections import OrderedDict
from ml4h.TensorMap import TensorMap, Interpretation

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

tf.get_logger().setLevel('ERROR')

In [ ]:
df = pd.read_csv(f'./all_split_32_v2022_05_10_meta_data.csv')
#df = pd.read_csv(f'./heroin_split_32_meta_data.csv')


step_size = 32

df = df[df.text.notna()]

df.psychoactive_class = df.psychoactive_class.apply(lambda x: x.strip())

drug2class = {d:i for i,d in enumerate(df.drug.unique())}
psychoactive2class = {d:i for i,d in enumerate(df.psychoactive_class.unique())}
ligand_chemical2class = {d:i for i,d in enumerate(df.ligand_chemical_class.unique())}
drug2weight = [(len(df)/len(df[df.drug==d])) for i,d in enumerate(df.drug.unique())]
print(psychoactive2class)
df['drug_class'] = [drug2class[d] for d in df.drug]
df['psychoactive_class_int'] = [psychoactive2class[d] for d in df.psychoactive_class]
psychoactive2weight = [(len(df)/len(df[df.psychoactive_class==d])) for d in df.psychoactive_class.unique()]
df['ligand_chemical_int'] = [ligand_chemical2class[d] for d in df.ligand_chemical_class]
chemical2weight = [(len(df)/len(df[df.ligand_chemical_class==d])) for d in df.ligand_chemical_class.unique()]
df[[f'tag_{i}' for i in range(52) ]] = df[[f'tag_{i}' for i in range(52) ]].fillna(0)
df[[f'tag_{i}' for i in range(52) ]] = df[[f'tag_{i}' for i in range(52) ]].astype(int)
tag2weight = {t:len(df)/sum(df[t]) for t in [f'tag_{i}' for i in range(52)] if sum(df[t])>100}

# train = df.sample(frac = 0.8)
# test = df.drop(train.index).sample(frac = 0.5)
# validate = df.drop(train.index).drop(test.index)

# df[df.set=='train'].to_csv('train.csv', index=False)
# df[df.set=='valid'].to_csv('valid.csv', index=False)
# df[df.set=='test'].to_csv('test.csv', index=False)

tags = {'Small_Group': 'tag_0', 'General': 'tag_1', 'First_Times': 'tag_2', 'Alone': 'tag_3', 'Difficult_Experiences': 'tag_4', 'Glowing_Experiences': 'tag_5', 'Retrospective_Summary': 'tag_6', 'Various': 'tag_7', 'Unknown_Context': 'tag_8', 'Mystical_Experiences': 'tag_9', 'Health_Problems': 'tag_10', 'Combinations': 'tag_11', 'Not_Applicable': 'tag_12', 'Bad_Trips': 'tag_13', 'Hangover_Days_After': 'tag_14', 'Entities_Beings': 'tag_15', 'Music_Discussion': 'tag_16', 'Addiction_Habituation': 'tag_17', 'Post_Trip_Problems': 'tag_18', 'Nature_Outdoors': 'tag_19', 'Relationships': 'tag_20', 'Depression': 'tag_21', 'Therapeutic_Intent_or_Outcome': 'tag_22', 'Overdose': 'tag_23', 'Medical_Use': 'tag_24', 'Sex_Discussion': 'tag_25', 'Train_Wrecks_Trip_Disasters': 'tag_26', 'Guides_Sitters': 'tag_27', 'Rave_Dance_Event': 'tag_28', 'Preparation_Recipes': 'tag_29', 'Festival_Lg_Crowd': 'tag_30', 'Health_Benefits': 'tag_31', 'Large_Group': 'tag_32', 'Multi-Day_Experience': 'tag_33', 'Club_Bar': 'tag_34', 'What_Was_in_That': 'tag_35', 'Personal_Preparation': 'tag_36', 'HPPD_Lasting_Visuals': 'tag_37', 'Families': 'tag_38', 'Second_Hand_Report': 'tag_39', 'Loss_of_Magic': 'tag_40', 'Hospital': 'tag_41', 'Public_Space': 'tag_42', 'School': 'tag_43', 'Poetry': 'tag_44', 'Performance_Enhancement': 'tag_45', 'Large_Party': 'tag_46', 'Group_Ceremony': 'tag_47', 'Workplace': 'tag_48', 'Cultivation_Synthesis': 'tag_49', 'Pregnancy_Baby': 'tag_50', 'Military': 'tag_51'}
itags = {v: k.replace('(', '').replace(')', '').replace('/', '') for k, v in tags.items()}
ctags = {v: int(k.replace('tag_', '')) for k, v in itags.items()}

receptors = ['5_ht2a', '5_ht2c', '5_ht2b', '5_ht1a', '5_ht1b', '5_ht1d', '5_ht1e', '5_ht1f', '5_ht3', '5_ht5a', '5_ht6', '5_ht7', 'dopamine_d1', 'dopamine_d2', 'dopamine_d3', 'dopamine_d4', 'dopamine_d5', 'adrenergic_alpha1a', 'adrenergic_alpha1b', 'adrenergic_alpha2a', 'adrenergic_alpha2b', 'adrenergic_beta1', 'adrenergic_beta2', 'sert', 'dat', 'net', 'imidazoline_1', 'sigma_1', 'sigma_2', 'dor', 'kor', 'mor', 'm1', 'm2', 'm3', 'm4', 'm5', 'h1', 'h2', 'h3', 'h4', 'calcium_channel', 'nmda', 'cb1', 'cb2', 'glutamate_ampa', 'gaba_a', 'gaba_b', 'dopamine_d2_long', 'dopamine_d2_short', 'sodium_channel', 'taar1', 'substance_p', 'paf_platelet_activating_factor', 'prostaglandin_e3', 'prostaglandin_e4', 'herg', 'monoamine_oxidase_a', 'monoamine_oxidase_b', 'cholecystokinin_a', 'cholecystokinin_b']
df[receptors] = df[receptors].astype(float)

In [ ]:
itags

In [ ]:
output_cols = [ 'psychoactive_class_int', 'ligand_chemical_int', 'drug_class']
output_cols += [f'tag_{i}' for i in range(35)]# if sum(df[f'tag_{i}']) > 100]


weighted_loss = False
tensor_maps_out = []
scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def weight_scce(weight_array):
    class_weights = tf.constant(weight_array)
    def my_loss(y_true, y_pred):
        weights = tf.gather(class_weights, y_true)
        return tf.compat.v1.losses.sparse_softmax_cross_entropy(y_true, y_pred, weights)
    return my_loss


def pearson(y_true, y_pred):
    # normalizing stage - setting a 0 mean.
    y_true -= K.mean(y_true, axis=-1)
    y_pred -= K.mean(y_pred, axis=-1)
    # normalizing stage - setting a 1 variance
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    # final result
    pearson_correlation = K.sum(y_true * y_pred, axis=-1)
    return pearson_correlation

for oc in output_cols:
    if 'drug_class' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,),
                                         loss=weight_scce(drug2weight) if weighted_loss else scce,
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'drug_{d}': v for d,v in drug2class.items()}))
    elif 'psychoactive_class_int' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,),
                                         loss=weight_scce(psychoactive2weight) if weighted_loss else scce,
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'{d}': v for d,v in psychoactive2class.items()}))
    elif 'ligand_chemical_int' == oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,),
                                         loss=weight_scce(chemical2weight) if weighted_loss else scce,
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'{d}': v for d,v in ligand_chemical2class.items()}))        
    elif 'tag_' in oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,), 
                                         loss=weight_scce([1.1, tag2weight[oc]]) if weighted_loss else scce,
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'no_{itags[oc]}': 0, f'{itags[oc]}': 1}))
    elif 'age' in oc:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CONTINUOUS, shape=(1,),
                                         loss=tf.keras.losses.MeanSquaredError(),
                                         metrics=[pearson]
                                         
                                        ))
    elif oc in receptors:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CONTINUOUS, shape=(1,),
                                         loss=tf.keras.losses.LogCosh(),
                                         metrics=[tf.metrics.MeanAbsoluteError()],))
    else:
        tensor_maps_out.append(TensorMap(f'{oc}', Interpretation.CATEGORICAL, shape=(1,), 
                                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.metrics.SparseCategoricalAccuracy()],
                                         channel_map={f'no_{oc}':0, f'{oc}':1}))


In [ ]:
epochs = 5
num_train_steps = 100 * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
classifier_model = tf.keras.models.load_model(f'./models/bertowid_weighted_scce', 
                                              custom_objects={'AdamWeightDecay':optimizer,
                                                              'my_loss': lambda x: x,
                                                               'pearson': pearson})

In [ ]:
classifier_model.summary()

In [ ]:
words = ['It was infuriating how they never listen to me',
         'I learned a lot I had never seen so deeply into the heart of god and the universe.', 
         'the desire is strong to repeat the experience on both smaller doses' ,
         'with music available for contemplation - at the beach - at night.',
         'There is complete recall of all experiences and thoughts, after the drug has been dissipated.',
         'The feeling of comfort, introspection, and tranquility',
         'I danced and raved all night long'
        ]

In [ ]:
r = classifier_model.predict(tf.constant(words))
drug_guess = tf.argmax(r[0], -1)
sigmoids = tf.nn.softmax([l for l in r if l.shape[-1] == 2])
print(drug_guess.numpy())
#original_results = tf.sigmoid(classifier_model(tf.constant(words)))

In [ ]:
int2drug = {i:d for i,d in enumerate(df.drug.unique())}
int2psychoactive = {i:d for i,d in enumerate(df.psychoactive_class.unique())}
int2chemical = {i:d for i,d in enumerate(df.ligand_chemical_class.unique())}

In [ ]:
for j,s in enumerate(words):
    for i,ot in enumerate(classifier_model.output_names):
        if 'tag' in ot:
            score = sigmoids[i-3, j, 1].numpy()
            score2 = tf.nn.softmax(r[i][j]).numpy()
            print(f'{s[:60]} <- {itags[ot]}: {score:0.2f}:')
        else:
            drug_guess = tf.argmax(r[i][j], -1).numpy()
            if r[i].shape[-1] == len(int2drug):
                print(f'drug guess: {drug_guess} is {int2drug[drug_guess]}')
            elif r[i].shape[-1] == len(int2psychoactive):
                print(f'Psychoactive guess: {drug_guess} is {int2psychoactive[drug_guess]}')
            elif r[i].shape[-1] == len(int2chemical):
                print(f'Chemical guess is {int2chemical[drug_guess]}')                

In [ ]:
[c for c in df]

In [ ]:
from collections import defaultdict
predictions = defaultdict(list)
truths = defaultdict(list)

def rec_dd():
    return defaultdict(rec_dd)
scores = rec_dd()

last_t = 2
max_testimonials = 150
max_drugs = 55
for index, row in df.iterrows():
    cur_t = row.testimonial# (row.testimonial,row.sentence)
    if last_t != cur_t and len(scores[row.drug]) >= max_testimonials and len(scores) >= max_drugs:
        break
    if last_t != cur_t and len(scores[row.drug]) >= max_testimonials:
        continue 
    
    results = classifier_model.predict(tf.constant([row.text]))
    results = [tf.nn.softmax(r).numpy() for r in results]
    if cur_t not in scores[row.drug]:
        for i,ot in enumerate(classifier_model.output_names):
            scores[row.drug][cur_t][ot] = []
    for i,ot in enumerate(classifier_model.output_names):
        scores[row.drug][cur_t][ot].append(results[i][0, 1])
        
    if len(scores[row.drug]) % 10 == 0 and last_t != cur_t:
        print(f'at {row.drug} with {len(scores[row.drug])} testimonials from {len(scores)} drug(s).')
    last_t = cur_t



In [ ]:
import pickle
# open file for writing
# create a binary pickle file 
f = open("split_32_bertowid_unweighted_max150_v2022_05_13.pkl","wb")
# # write the python object (dict) to pickle file
pickle.dump(scores,f)

In [ ]:
import pickle
def rec_dd():
    return defaultdict(rec_dd)
#scores = rec_dd()

scores = pickle.load(open("split_32_bertowid_unweighted_max150_v2022_05_13.pkl", "rb"))

In [ ]:
[s for s in scores]

In [ ]:
def get_mean_trajectories(scores, column='psychoactive_class', new_size = 0, out_col = "tag_0"):
    standardized_psycho = rec_dd()
    means_psycho = Counter()
    stds_psycho = Counter()
    new_sizes = {}
    for drug in scores:
        lens = 0
        for t in scores[drug]:
            lens += len(scores[drug][t][out_col])
        if new_size <= 0:
            new_sizes[drug] = int(lens/len(scores[drug]))
        else:
            new_sizes[drug] = new_size
            
        print(f'Drug {drug} from {len(scores[drug])} testimonials, mean length: {lens/len(scores[drug]):0.1f} Interpt to: { new_sizes[drug]:0.0f}')
    print(output_cols)
    for drug in scores:
        psychoactive_class = df[df.drug==drug][column].unique()[0]
        for t in scores[drug]:
            for s in scores[drug][t]:
                try:
                    interpolator = interp.interp1d(np.arange(len(scores[drug][t][s])), scores[drug][t][s])

                    standardized_psycho[drug][t][s] = interpolator(np.linspace(0, len(scores[drug][t][s])-1, new_sizes[drug]))
                    means_psycho[psychoactive_class,s] += standardized_psycho[drug][t][s]
                    stds_psycho[psychoactive_class,s] += standardized_psycho[drug][t][s]*standardized_psycho[drug][t][s]
                except ValueError as e:
                    print(f'vall err {drug} {t} len: {len(scores[drug][t][s])} e:{e}')
                    break

    for pc in df[column].unique():
        n = 1
        for drug in df[df[column]==pc].drug.unique():
            n += len(standardized_psycho[drug])
        print(f'Class {pc} has length: {n}')
        for s in output_cols:
            means_psycho[pc,s] /= n
            stds_psycho[pc,s] /= n
            stds_psycho[pc,s] -= means_psycho[pc,s]*means_psycho[pc,s]
            stds_psycho[pc,s] = np.sqrt(stds_psycho[pc,s])
    return standardized_psycho, means_psycho, stds_psycho

def plot_tag_trajectories(labels, means, stds, keep_drugs, keep_tags,
                   figsize=(42, 42), cols=5, rows=4, dpi=300, ncol=4):
    fig, axes = plt.subplots(rows, cols, figsize=figsize, dpi=dpi)
    cmap = plt.get_cmap('hsv')
    colors = cmap(np.linspace(0, 1, len(labels)))
    markers = "v.,o1^2>348<spdxh*PH+XD|"
    o_drugs = sorted(labels)
    o_tags = [ot for ot in classifier_model.output_names if 'tag_' in ot and itags[ot] in keep_tags]

    for sentiment, ax in zip(o_tags, axes.ravel()):
        for i, (drug, color) in enumerate(zip(o_drugs, colors)):
            if drug not in keep_drugs:
                continue
            ax.plot(range(len(means[drug,sentiment])), means[drug,sentiment], 
                    c=color, marker=markers[i%len(markers)], 
                    label=drug.capitalize() if len(drug) > 4 else drug.upper(),
                    fillstyle='none' if i%2==0 else 'full')
            ax.fill_between(range(len(means[drug,sentiment])), 
                            means[drug,sentiment] - stds[drug,sentiment]/2, 
                            means[drug,sentiment] + stds[drug,sentiment]/2, 
                            color=color, alpha=0.05)
        ax.set_title(f'{itags[sentiment].replace("_", " ")} Trajectory of Trip')
        ax.set_xlabel('Time in Narrative')
        ax.set_ylabel(f'{itags[sentiment].replace("_", " ")}')
        ax.set_xticks(())
        ax.set_yticks(())
        ax.legend(ncol=ncol)
    plt.tight_layout()



def get_drug_tag_scape(means, labels):
    avg = defaultdict(list)
    drug_order = {}
    for drug1,sentiment1 in means:
        print(f'\nat sentiment {sentiment1} d:{drug1} ')
        if 'tag' not in sentiment1 or isinstance(means[drug1,sentiment1], float):
            continue
        avg[sentiment1].append(means[drug1,sentiment1].mean())
        drug_order[drug1] = len(drug_order)-1
    order_drug = {v:k for k,v in drug_order.items()}
    
    drug_sentiments = defaultdict(dict)
    for sentiment in avg:
        print(f'\nat sentiment {sentiment} np.argsort(avg[sentiment]):{np.argsort(avg[sentiment]):} ')
        for i, idx in enumerate(np.argsort(avg[sentiment])):
            #print(f'{order_drug[idx]} has {idx}'  )
            drug_sentiments[order_drug[idx]][sentiment] = i - (len(order_drug)/2)
    print(f'\nat sentiment {drug_sentiments["entactogen"]} ')
    return drug_sentiments
        

def plot_tag_drug(title, agrees, figsize=(2, 8)):
    _ = plt.figure(figsize=figsize, dpi=300)
    my_cmap = plt.get_cmap("bwr_r")
    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
    plt.barh(range(len(agrees)), [t[1] for t in agrees], color=my_cmap(rescale([t[1] for t in agrees])))
    plt.axvline(0.0, linestyle='dashed', c='orange')
    plt.yticks(np.arange(len(agrees)), [' '.join(itags[t[0]].split('_')[:2]) for t in agrees], ha='right')
    plt.title(title)
    plt.xticks(())
    plt.xlabel('Relative Ranking')
    plt.box(False)
    
    




list(tags.keys())

keep_drugs = list(scores.keys())
keep_drugs = ['5-meo-dmt', 'dmt', 'psilocin', 'mdma',  'hydrocodone', 'sertraline', 'cocaine',
             'venlafaxine', 'methamphetamine', 'hydrocodone', 'morphine', 'oxycodone', 'salvia', 'ketamine',
              'lsd', 'ibogaine', 'cocaine', 'amphetamine',
             '2c-b','2c-i', 'methadone', 'haloperidol']
# keep_drugs = [ 'sertraline',  'paroxetine',
#              'venlafaxine', 'methamphetamine', 'cocaine', 'amphetamine',]

keep_tags = [
    'Mystical_Experiences', 'Entities_Beings',
   # 'Addiction_Habituation', 'Depression' ,  #'Combinations', 'Bad_Trips', 'Various', "Unknown_Context", 'Retrospective_Summary'
    
            ]
        
        # mdma_tags = [
#              'Rave_Dance_Event', 'Music_Discussion', 'Festival_Lg_Crowd','Club_Bar', 'Large_Group','Relationships', 'Sex_Discussion', 
#     'Glowing_Experiences', 
        #list(tags.keys()) # ['Rave_Dance_Event']
#keep_tags = ['Rave_Dance_Event', 'Mystical_Experiences', 'Depression' ]
# 'Mystical_Experiences', 'Entities_Beings', 'Depression', 
# plot_tag_trajectories(scores, means, stds, keep_drugs, keep_tags, 
#                       figsize=(9, len(keep_tags)*2), cols=1, rows=len(keep_tags), ncol=6)
plot_tag_trajectories(list(scores.keys()), means_d, stds_d, keep_drugs, keep_tags, 
                      figsize=(9.25, len(keep_tags)*4), cols=1, rows=len(keep_tags), ncol=5)

standardized_d, means_d, stds_d = get_mean_trajectories(scores, column='drug')
standardized_p, means_p, stds_p = get_mean_trajectories(scores, column='psychoactive_class')

d_sentiments = get_drug_tag_scape(means_d, df.drug.unique())

p_sentiments = get_drug_tag_scape(means_p, df.psychoactive_class.unique())

for d in p_sentiments:
    ttt = sorted(p_sentiments[d].items(), key = lambda x: x[1])
    plot_tag_drug(d.capitalize() if len(d) > 5 else d.upper(), ttt, figsize=(0.8, 8))

for d in d_sentiments:
    ttt = sorted(d_sentiments[d].items(), key = lambda x: x[1])
    plot_tag_drug(d.capitalize() if len(d) > 5 else d.upper(), ttt, figsize=(0.5, 8))

In [ ]:
ttt = sorted(d_sentiments["venlafaxine"].items(), key = lambda x: x[1])
plot_tag_drug('Venlafaxine', ttt, figsize=(1.1, 7))

In [ ]:
ttt = sorted(p_sentiments["antipsychotic"].items(), key = lambda x: x[1])
plot_tag_drug('Antipsychotic', ttt)

In [ ]:
ttt = sorted(p_sentiments["psychedelic"].items(), key = lambda x: x[1])
plot_tag_drug('Psychedelic', ttt, figsize=(1.2, 7))

In [ ]:
ttt = sorted(p_sentiments["antidepressant"].items(), key = lambda x: x[1])
plot_tag_drug('Antidepressant', ttt)

In [ ]:
ttt = sorted(p_sentiments["hallucinogen"].items(), key = lambda x: x[1])
plot_tag_drug('Hallucinogen', ttt)

In [ ]:
ttt = sorted(p_sentiments["stimulant"].items(), key = lambda x: x[1])
plot_tag_drug('Stimulant', ttt)

In [ ]:
keep_tags = list(tags.keys()) # ['Rave_Dance_Event']
keep_drugs = list(scores.keys())
plot_tag_trajectories(df.psychoactive_class.unique(), means_p, stds_p, df.psychoactive_class.unique(), keep_tags, 
                      figsize=(9, len(keep_tags)*4), cols=1, rows=len(keep_tags), ncol=3)

In [ ]:
[c for c in df]

In [ ]:
standardized_c, means_c, stds_c = get_mean_trajectories(scores, column='ligand_chemical_class')

In [ ]:
keep_tags = list(tags.keys()) # ['Rave_Dance_Event']
keep_drugs = list(scores.keys())
plot_tag_trajectories(df.ligand_chemical_class.unique(), means_c, stds_c, 
                      df.ligand_chemical_class.unique(), keep_tags, 
                      figsize=(12, 35*4), cols=1, rows=35, ncol=5)

In [ ]:
!pip install fastdtw

from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
def get_dtw(means, labels):
    distances = {} 
    idxs = Counter()
    jdxs = Counter()

    for drug1,sentiment1 in means:
    #     if 'love' not in sentiment1:
    #         continue
        if sentiment1 not in distances:
            distances[sentiment1] = np.zeros((len(labels),len(labels)))
        for drug2,sentiment2 in means:
            if sentiment1 != sentiment2:
                continue
            distance, path = fastdtw(means[drug1,sentiment1], 
                                     means[drug2,sentiment2], 
                                     dist=euclidean)
            if means[drug1,sentiment1].mean() > means[drug2,sentiment2].mean():
                sign = 1
            else:
                sign = -1
            distances[sentiment1][idxs[sentiment1], jdxs[sentiment1]] = distance * sign
            jdxs[sentiment1]+= 1
        idxs[sentiment1]+=1
        jdxs[sentiment1]=0
        print(f'Finished Drug {drug1} of {len(means)} ')
    return distances
def confusion_heatmap(confusion, labels, figsize=(12, 8), 
                      title='Emotion Heatmap'):
    fig, ax = plt.subplots(figsize=figsize, dpi=300)
    ax = sb.heatmap(confusion, cmap='bwr', cbar_kws={"shrink": .8}, cbar=True, ax=ax)

    ax.set_title(title)
    ax.set_xticks(np.arange(confusion.shape[0]) + 0.5)
    ax.set_yticks(np.arange(confusion.shape[0]) + 0.5)
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([np.min(confusion), np.max(confusion)])
    cbar.set_ticklabels([f'Less \n{title.split()[0].capitalize()}', 
                         f'More \n{title.split()[0].capitalize()}'])
    labels_format = [l.capitalize() if len(l) > 5 else l.upper() for l in labels]
    ax.set_xticklabels(labels=labels_format, ha='right', rotation=45)
    ax.set_yticklabels(labels=labels_format, rotation=0)
    plt.tight_layout()
    figure_path='./confusion_heatmap.png'
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    plt.show()

In [ ]:
def confusion_heatmap(confusion, labels, figsize=(12, 8), 
                      title='Emotion Heatmap'):
    fig, ax = plt.subplots(figsize=figsize, dpi=300)
    ax = sb.heatmap(confusion, cmap='bwr', cbar_kws={"shrink": .8}, cbar=True, ax=ax)

    ax.set_title(title)
    ax.set_xticks(np.arange(confusion.shape[0]) + 0.5)
    ax.set_yticks(np.arange(confusion.shape[0]) + 0.5)
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([np.min(confusion), np.max(confusion)])
    cbar.set_ticklabels([f'Less \n{title.split()[0].capitalize()}', 
                         f'More \n{title.split()[0].capitalize()}'])
    labels_format = [l.capitalize() if len(l) > 5 else l.upper() for l in labels]
    ax.set_xticklabels(labels=labels_format, ha='right', rotation=45)
    ax.set_yticklabels(labels=labels_format, rotation=0)
    plt.tight_layout()
    figure_path='./confusion_heatmap.png'
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    plt.show()

In [ ]:
import seaborn as sb

In [ ]:
distances = get_dtw(means_d, df.drug.unique())

In [ ]:
distances['tag_0'].shape

In [ ]:
confusion_heatmap(distances['tag_0'], df.drug.unique(), title=f'{itags["tag_0"]} DTW')

In [ ]:
for sentiment in distances:
    if sentiment not in itags:
        continue
    confusion_heatmap(distances[sentiment], df.drug.unique(), figsize=(16, 12),
                      title=f"{itags[sentiment].replace('_', ' ')} Dynamic Time Warps Between Drugs") 